In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2024-02-19 03:10:58.880776: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 03:10:58.880880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 03:10:59.028789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_33/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


In [4]:
'''df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=':::',names=['ID','TITLE','GENRE','DESCRIPTIONS'])
display(train_data.head())
print(test_data.shape)'''

'df_test=pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt", sep=\':::\',names=[\'ID\',\'TITLE\',\'GENRE\',\'DESCRIPTIONS\'])\ndisplay(train_data.head())\nprint(test_data.shape)'

In [5]:
X = df['DESCRIPTION'].values
y = df['GENRE'].values

In [6]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [7]:
df.isna().sum()

ID             0
TITLE          0
GENRE          0
DESCRIPTION    0
dtype: int64

Data pre processing

#Step 1 converting into lower cases, so that two words with such as Ball and ball will be considered as singel token


In [12]:
df['DESCRIPTION']=df['DESCRIPTION'].str.lower()

In [13]:
df['DESCRIPTION']

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the film's title refers not only to the un-re...
                               ...                        
54209     this short-lived nbc live sitcom centered on ...
54210     the next generation of exploitation. the sist...
54211     ze bestaan echt, is a stand-up comedy about g...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend, 1935, the most intense ...
Name: DESCRIPTION, Length: 54214, dtype: object

Removing html tags if there are any

In [19]:
df['DESCRIPTION']=df['DESCRIPTION'].apply(remove_html)

In [16]:
import re
def remove_html(text):
    pattern=re.compile('<,*?>')
    return pattern.sub('r',text)

In [ ]:
'''
def remove_url(text):
    pattern=re.compile(r'')
'''


Removing punctuation 


In [21]:
import string, time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
exclude=string.punctuation

In [24]:
'''
def remove_punctuation(text):
    for char in exclude:
        text=text.replace(char,'')
    return text
'''

In [25]:
df['DESCRIPTION']=df['DESCRIPTION'].apply(remove_punctuation)

The function written above takes longer time to execute so we would use the standard practive for removing punctuation from the text

In [27]:
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))

In [29]:
df['DESCRIPTION']=df['DESCRIPTION'].apply(remove_punc)

Removing chatwords

Spelling correction 

In [31]:
from textblob import TextBlob 

In [35]:
#df['DESCRIPTION']=df['DESCRIPTION'].apply(lambda x: str(TextBlob(x).correct()))
#time consuming so we will skip for now

Removing stopwords

In [36]:
from nltk.corpus import stopwords

In [39]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [40]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)

In [ ]:
df['DESCRIPTION']=df['DESCRIPTION'].apply(remove_stopwords)

In [43]:
df['DESCRIPTION']

0         listening in to a conversation between his do...
1         a brother and sister with a past incestuous r...
2         as the bus empties the students for their fie...
3         to help their unemployed father make ends mee...
4         the films title refers not only to the unreco...
                               ...                        
54209     this shortlived nbc live sitcom centered on b...
54210     the next generation of exploitation the siste...
54211     ze bestaan echt is a standup comedy about gro...
54212     walter and vivian live in the country and hav...
54213     on labor day weekend 1935 the most intense hu...
Name: DESCRIPTION, Length: 54214, dtype: object

Removing stopwords using NLTK


In [44]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
#df['DESCRIPTION']=df['DESCRIPTION'].apply(word_tokenize)

Using spacy for tokeniziation as it will give the best results

In [ ]:
import spacy 
nlp=spacy.load('en_core_web_sm')

In [ ]:
df['DESCRIPTION']=df['DESCRIPTION'].apply(lambda x:[token.text for token in nlp(x)])